In [2]:
import random

# Hi-Lo card counting value assignment
def card_value(card):
    if card in [2, 3, 4, 5, 6]:
        return 1
    elif card in [7, 8, 9]:
        return 0
    else:
        return -1

# Make a fresh, shuffled one-deck shoe
def fresh_deck():
    deck = [2, 3, 4, 5, 6, 7, 8, 9, 10, 10, 10, 10, 11] * 4  # 10 = J/Q/K, 11 = Ace
    random.shuffle(deck)
    return deck

def deal_card(deck):
    return deck.pop()

def should_hit(dealer_upcard, player_total, count, use_card_counting=False):
    # If using card counting: Stand on 15+ when the count is high
    threshold = 17
    if use_card_counting and count >= 2:
        threshold = 15
    return player_total < threshold

def play_game(use_card_counting=False):
    deck = fresh_deck()
    count = 0

    # Deal initial hands
    player_hand = [deal_card(deck), deal_card(deck)]
    dealer_hand = [deal_card(deck), deal_card(deck)]

    # Count visible cards (player hand + dealer upcard)
    for card in player_hand + [dealer_hand[0]]:
        val = 10 if card == 11 else card
        count += card_value(val)

    player_total = sum(player_hand)
    dealer_upcard = dealer_hand[0]

    # --- Player's turn ---
    while should_hit(dealer_upcard, player_total, count, use_card_counting):
        new_card = deal_card(deck)
        val = 10 if new_card == 11 else new_card
        count += card_value(val)
        player_hand.append(new_card)
        player_total = sum(player_hand)
        # If bust and there's an ace (11), turn ace into 1 and retry
        while player_total > 21 and 11 in player_hand:
            ace_index = player_hand.index(11)
            player_hand[ace_index] = 1
            player_total = sum(player_hand)
        if player_total > 21:
            return 0  # Player busts, loses

    # --- Dealer's turn ---
    dealer_total = sum(dealer_hand)
    while dealer_total < 17:
        new_card = deal_card(deck)
        dealer_hand.append(new_card)
        dealer_total = sum(dealer_hand)
        while dealer_total > 21 and 11 in dealer_hand:
            ace_index = dealer_hand.index(11)
            dealer_hand[ace_index] = 1
            dealer_total = sum(dealer_hand)
        if dealer_total > 21:
            return 1  # Dealer busts, player wins

    # --- Compare final hands ---
    if player_total > dealer_total:
        return 1  # Player wins
    else:
        return 0  # Player loses or ties

def simulate_many_games(n=10000):
    classic_wins = sum(play_game(False) for _ in range(n))
    count_wins = sum(play_game(True) for _ in range(n))
    print(f"Classic strategy win rate: {classic_wins/n:.4f}")
    print(f"Card counting strategy win rate: {count_wins/n:.4f}")

if __name__ == "__main__":
    simulate_many_games(n=10000)


Classic strategy win rate: 0.4087
Card counting strategy win rate: 0.4150
